In [1]:
import torch 
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
input_size = 28
time_steps = 28
num_layers = 2
hidden_size = 256
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 2

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [17]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*time_steps, num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        print(x.shape)
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [18]:
x = torch.rand(10, 10, 30)
model = NN(30, 10)

In [19]:
model(x).shape

torch.Size([10, 10, 10])

In [21]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

In [22]:
x = torch.rand(64, 1, 28, 28)
x = x.squeeze(1)
print(x.shape)
model(x).shape

torch.Size([64, 28, 28])
torch.Size([64, 28, 28])


torch.Size([64, 10])

In [20]:
train_dataset = datasets.MNIST(root='.', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='.', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [8]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [9]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.squeeze(1)
#         data = data.reshape(data.shape[0], -1)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

100%|██████████| 938/938 [02:28<00:00,  6.33it/s]


In [13]:
def check_acc(loader, model):
    if loader.dataset.train:
        print('checking the accuracy on training data')
    else: 
        print('checking the accuracy on test data')
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.squeeze(1)
#             x = x.reshape(x.shape[0], -1)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print('{}/{} with accuracy {}'.format(num_correct, num_samples, float(num_correct)/float(num_samples)))
    
    model.train()

In [14]:
check_acc(train_loader, model)

  0%|          | 2/938 [00:00<00:47, 19.78it/s]

checking the accuracy on training data


100%|██████████| 938/938 [00:44<00:00, 20.98it/s]

59223/60000 with accuracy 0.98705


In [15]:
check_acc(test_loader, model)

  1%|▏         | 2/157 [00:00<00:07, 19.73it/s]

checking the accuracy on test data


100%|██████████| 157/157 [00:07<00:00, 21.46it/s]

9860/10000 with accuracy 0.986


In [16]:
for batch_idx, (data, target) in enumerate(train_loader):
    print(data.shape)
    break

torch.Size([64, 1, 28, 28])
